<a href="https://colab.research.google.com/github/shiragelb/NCC-Statistical-Reports/blob/main/mini_pipline_concise_(2)_(3).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pdfplumber
!pip install camelot-py[cv]
!pip install tabula-py
!pip install python-docx
!pip install anthropic
!pip install plotly -q
!pip install networkx
!pip install scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 101.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.0/308.0 kB 24.6 MB/s eta 0:00:00


In [2]:
import requests
import os
from docx import Document
import pandas as pd
from google.colab import files
import camelot
import tabula
import pdfplumber
from docx.shared import Inches # Import Inches for setting image size
import json
from datetime import datetime

What This System Does:
1. Initial Data Extraction (First Part)

Takes uploaded DOCX files (chapters from 2001 and 2002)
Extracts tables from these documents
Saves them as CSV files with a naming pattern: table{i}{chapter}{year}.csv
Creates a reference dictionary mapping Hebrew headers to file paths

2. Core Functionality - Table Evolution Tracking
The system solves this problem: "Given tables from different years, which tables in Year 2 are continuations of tables from Year 1?"
This is challenging because:

Tables may change headers slightly year-to-year
Hebrew text requires special processing
Tables might split (1→N) or merge (N→1)
Some tables may disappear and reappear (gaps)

3. Processing Pipeline
DOCX Files → Table Extraction → Hebrew Processing → Embeddings → Similarity Matching → Chain Building → Validation → Visualization
Key Steps:

Hebrew Text Processing: Normalizes Hebrew text, removes year references, handles special characters
Embedding Generation: Creates semantic vectors for each table using multilingual models
Similarity Computation: Builds matrices comparing all tables between years
Hungarian Algorithm: Finds optimal 1-to-1 matches between years
Special Cases Detection: Identifies splits, merges, and gaps
Claude API Validation: For uncertain matches (0.85-0.97 similarity), asks Claude to validate
Chain Building: Creates continuous chains showing table evolution

How It Relies on Initial Data:
The initial DOCX uploads provide:

Table Content: The actual data and headers
Structure: How many tables exist each year
Hebrew Headers: Critical for matching - the system must understand that "מספר הילדים לפי גיל 2001" and "מספר הילדים לפי גיל 2002" are the same table

The process_documents() function creates:

CSV files in tables/ directory
table_references.json with Hebrew header → filename mappings

Key Thresholds:

>0.97: Automatic match (high confidence)
0.85-0.97: Edge case (needs API validation)
<0.85: No match
Splits/Merges: Detected at 0.80+ similarity

Output:

JSON graph structure showing table evolution
HTML/Markdown reports with statistics
Interactive Sankey diagrams
Validation reports

The code is essentially building a temporal knowledge graph of how statistical tables evolve, handling the complexities of real-world data where tables don't always continue cleanly from year to year.

In [3]:
from google.colab import files

# This opens a file dialog to select files
uploaded = files.upload()

Saving tables_columns.json to tables_columns.json
Saving tables_summary.json to tables_summary.json


#  config.py

In [4]:
%%writefile config.py
import json
from dataclasses import dataclass
from typing import Optional


@dataclass
class MatchingConfig:
    tables_dir: str = "/content/tables"  # Updated from "tables"
    reference_json: str = "tables_summary.json"  # Updated from "table_references.json"
    columns_json: str = "tables_columns.json"  # New field for column headers
    output_dir: str = "output"

    similarity_threshold: float = 0.85
    confident_threshold: float = 0.97
    split_threshold: float = 0.80
    merge_threshold: float = 0.80
    max_gap_years: int = 2

    use_api_validation: bool = False
    api_key: Optional[str] = None

    def save(self, path="config.json"):
        with open(path, 'w') as f:
            json.dump(self.__dict__, f, indent=2)

Writing config.py


# hebrew_processor.py

In [5]:
%%writefile hebrew_processor.py
import re
import unicodedata

class HebrewProcessor:
    def __init__(self):
        self.year_patterns = [
            r'ממוצע \d{4}', r'סוף \d{4}', r'\d{4}'
        ]

    def process_header(self, text):
        text = unicodedata.normalize('NFC', text)
        text = re.sub(r'[\u0591-\u05C7]', '', text)
        for pattern in self.year_patterns:
            text = re.sub(pattern, '', text)
        text = re.sub(r'\(המשך\)', '', text)
        text = re.sub(r'לוח:\s*\d+\.\d+', '', text)
        return ' '.join(text.split()).strip()

Writing hebrew_processor.py


# table_loader.py

In [32]:
%%writefile table_loader.py
import os
import json
import pandas as pd
import re


class TableLoader:
    def __init__(self, tables_dir="/content/tables",
                 reference_json="tables_summary.json",
                 columns_json="tables_columns.json"):
        self.tables_dir = tables_dir
        self.reference_json = reference_json
        self.columns_json = columns_json
        self.tables_metadata = {}
        self.tables_by_year = {}
        self.column_names = {}  # New: store column names

    def load_metadata(self):
        # Load the main tables summary (identifier → header)
        with open(self.reference_json, 'r', encoding='utf-8') as f:
            identifier_to_header = json.load(f)

        # Load column names if available
        if os.path.exists(self.columns_json):
            with open(self.columns_json, 'r', encoding='utf-8') as f:
                self.column_names = json.load(f)

        # Process each identifier
        for identifier, header in identifier_to_header.items():
            # Parse new format: serial_chapter_year (e.g., "1_03_2021")
            match = re.match(r'(\d+)_(\d+)_(\d{4})', identifier)
            if match:
                serial, chapter, year = match.groups()
                serial = int(serial)
                chapter_str = chapter  # Keep as string with leading zeros
                chapter_int = int(chapter)
                year = int(year)

                # Build filepath (not used but kept for metadata)
                filepath = os.path.join(self.tables_dir, str(year),
                                    chapter_str, f"{identifier}.csv")

                # Clean column names
                clean_cols = self.clean_column_names()

                # Store metadata with identifier as key
                self.tables_metadata[identifier] = {
                    'id': identifier,
                    'file': filepath,
                    'header': header,
                    'year': year,
                    'chapter': chapter_int,
                    'serial': serial,
                    'columns': self.clean_cols.get(identifier, [])  # Add column names
                }

                # Group by year
                if year not in self.tables_by_year:
                    self.tables_by_year[year] = []
                self.tables_by_year[year].append(identifier)

        return len(self.tables_metadata)

    def load_table_data(self, table_id):
        """Load actual CSV data for a table"""
        metadata = self.tables_metadata.get(table_id)
        if metadata and os.path.exists(metadata['file']):
            return pd.read_csv(metadata['file'], header=None)
        return None

    def get_header_for_identifier(self, identifier):
        """Get header text for an identifier"""
        metadata = self.tables_metadata.get(identifier)
        return metadata['header'] if metadata else None

    def get_columns_for_identifier(self, identifier):
        """Get column names for an identifier"""
        metadata = self.tables_metadata.get(identifier)
        return metadata['columns'] if metadata else []

    def clean_column_names(self):
        """
        Clean column names by removing float patterns and '--' markers
        while preserving integers and special characters.
        """
        # Define regex patterns for removable content
        patterns_to_remove = [
            # Float numbers with decimal point (e.g., "19.0", "23.7")
            r'\d+\.\d+',

            # Numbers with thousand separators and decimals (e.g., "1,183.3")
            r'\d{1,3}(?:,\d{3})+\.\d+',

            # Numbers with thousand separators without decimals (e.g., "1,183" or "131,936")
            r'\d{1,3}(?:,\d{3})+',

            # Double dashes
            r'--'
        ]

        # Combine all patterns into one regex
        combined_pattern = '|'.join(patterns_to_remove)

        # Process each identifier's column list
        for identifier, columns in self.column_names.items():
            cleaned_columns = []

            for column in columns:
                if column is None:
                    cleaned_columns.append(column)
                    continue

                # Remove all matching patterns
                cleaned = re.sub(combined_pattern, '', column)

                # Clean up extra spaces
                cleaned = ' '.join(cleaned.split())

                # Strip leading/trailing spaces
                cleaned = cleaned.strip()

                cleaned_columns.append(cleaned)

            # Update the column names with cleaned version
            self.column_names[identifier] = cleaned_columns

        # Save the cleaned data to a new JSON file
        with open('column_names_cleaned.json', 'w', encoding='utf-8') as f:
            json.dump(self.column_names, f, ensure_ascii=False, indent=2)

        print("Column names cleaned and saved to 'column_names_cleaned.json'")

Overwriting table_loader.py


# Similarity Matrix

In [7]:
%%writefile similarity.py
import numpy as np
from scipy.spatial.distance import cosine

class SimilarityBuilder:
    def compute_similarity_matrix(self, chain_embeddings, table_embeddings):
        chain_ids = list(chain_embeddings.keys())
        table_ids = list(table_embeddings.keys())

        n_chains = len(chain_ids)
        n_tables = len(table_ids)

        matrix = np.zeros((n_chains, n_tables))

        for i, chain_id in enumerate(chain_ids):
            for j, table_id in enumerate(table_ids):
                # Cosine similarity
                sim = 1 - cosine(chain_embeddings[chain_id],
                                 table_embeddings[table_id])
                matrix[i, j] = (sim + 1) / 2  # Normalize to [0,1]

        return {
            'matrix': matrix,
            'chain_ids': chain_ids,
            'table_ids': table_ids
        }

Writing similarity.py


# Hungarian Matching

In [8]:
%%writefile hungarian.py
from scipy.optimize import linear_sum_assignment

class HungarianMatcher:
    def __init__(self, threshold=0.85):
        self.threshold = threshold

    def find_optimal_matching(self, sim_matrix):
        matrix = sim_matrix['matrix']
        chain_ids = sim_matrix['chain_ids']
        table_ids = sim_matrix['table_ids']

        # Convert to cost matrix
        cost = 1 - matrix
        row_ind, col_ind = linear_sum_assignment(cost)

        matches = []
        for i, j in zip(row_ind, col_ind):
            if i < len(chain_ids) and j < len(table_ids):
                similarity = matrix[i, j]
                if similarity >= self.threshold:
                    matches.append((chain_ids[i], table_ids[j], similarity))

        unmatched_chains = [c for i, c in enumerate(chain_ids)
                           if i not in row_ind]
        unmatched_tables = [t for j, t in enumerate(table_ids)
                           if j not in col_ind]

        return {
            'matches': matches,
            'unmatched_chains': unmatched_chains,
            'unmatched_tables': unmatched_tables
        }

Writing hungarian.py


# Split/Merge Detection

In [9]:
%%writefile split_merge.py
class SplitMergeDetector:
    def __init__(self, split_threshold=0.80, merge_threshold=0.80):
        self.split_threshold = split_threshold
        self.merge_threshold = merge_threshold

    def detect_splits(self, sim_matrix):
        splits = []
        matrix = sim_matrix['matrix']
        chain_ids = sim_matrix['chain_ids']
        table_ids = sim_matrix['table_ids']

        for i, chain_id in enumerate(chain_ids):
            high_sim_tables = []
            for j, table_id in enumerate(table_ids):
                if matrix[i, j] >= self.split_threshold:
                    high_sim_tables.append((table_id, matrix[i, j]))

            if len(high_sim_tables) >= 2:
                splits.append({
                    'chain': chain_id,
                    'targets': high_sim_tables
                })

        return splits

    def detect_merges(self, sim_matrix):
        merges = []
        matrix = sim_matrix['matrix']
        chain_ids = sim_matrix['chain_ids']
        table_ids = sim_matrix['table_ids']

        for j, table_id in enumerate(table_ids):
            high_sim_chains = []
            for i, chain_id in enumerate(chain_ids):
                if matrix[i, j] >= self.merge_threshold:
                    high_sim_chains.append((chain_id, matrix[i, j]))

            if len(high_sim_chains) >= 2:
                merges.append({
                    'table': table_id,
                    'sources': high_sim_chains
                })

        return merges

Writing split_merge.py


# Chain Manager

In [10]:
%%writefile chains.py
from collections import defaultdict


class ChainManager:
    def __init__(self):
        self.chains = {}
        self.match_details = {}  # Store similarity scores and API usage

    def initialize_from_first_year(self, tables):
        for table_id, metadata in tables.items():
            chain_id = f"chain_{table_id}"
            self.chains[chain_id] = {
                'id': chain_id,
                'tables': [table_id],
                'years': [metadata['year']],
                'headers': [metadata['header']],
                'columns': [metadata.get('columns', [])],  # New: track columns
                'status': 'active',
                'gaps': [],
                'similarities': [],  # Store similarity scores
                'api_validated': []  # Track API validation usage
            }
        return len(self.chains)

    def update_chains(self, matches, year, table_metadata, api_validations=None):
        matched_chains = set()
        for match_info in matches:
            # Handle both tuple and dict formats
            if isinstance(match_info, tuple):
                chain_id, table_id, similarity = match_info
                api_used = False
            else:
                chain_id = match_info['chain_id']
                table_id = match_info['table_id']
                similarity = match_info['similarity']
                api_used = match_info.get('api_validated', False)

            if chain_id in self.chains:
                self.chains[chain_id]['tables'].append(table_id)
                self.chains[chain_id]['years'].append(year)
                self.chains[chain_id]['similarities'].append(similarity)
                self.chains[chain_id]['api_validated'].append(api_used)

                if table_id in table_metadata:
                    self.chains[chain_id]['headers'].append(table_metadata[table_id]['header'])
                    # New: Add column information
                    self.chains[chain_id]['columns'].append(table_metadata[table_id].get('columns', []))

                # Store match details for visualization
                edge_key = f"{self.chains[chain_id]['tables'][-2]}_{table_id}"
                self.match_details[edge_key] = {
                    'similarity': similarity,
                    'api_validated': api_used
                }

                matched_chains.add(chain_id)

        # Mark unmatched as dormant
        for chain_id, chain in self.chains.items():
            if chain['status'] == 'active' and chain_id not in matched_chains:
                chain['status'] = 'dormant'
                chain['gaps'].append(year)

    def get_chain_embeddings(self, embeddings_dict):
        chain_embeddings = {}
        for chain_id, chain in self.chains.items():
            if chain['status'] == 'active' and chain['tables']:
                last_table = chain['tables'][-1]
                if last_table in embeddings_dict:
                    chain_embeddings[chain_id] = embeddings_dict[last_table]
        return chain_embeddings

    def get_columns_for_chain(self, chain_id):
        """Get all column sets in a chain"""
        if chain_id in self.chains:
            return self.chains[chain_id].get('columns', [])
        return []

Writing chains.py


# Report Generator

In [11]:
%%writefile report_gen.py
import json
import numpy as np
from datetime import datetime


class ReportGenerator:
    def __init__(self):
        self.timestamp = datetime.now()

    def generate_summary(self, chains, statistics):
        """Generate summary with column statistics"""
        # Calculate column statistics
        total_column_sets = sum(len(chain.get('columns', [])) for chain in chains.values())
        avg_columns_per_table = 0
        if total_column_sets > 0:
            all_column_counts = []
            for chain in chains.values():
                for cols in chain.get('columns', []):
                    if cols:
                        all_column_counts.append(len(cols))
            if all_column_counts:
                avg_columns_per_table = sum(all_column_counts) / len(all_column_counts)

        summary = {
            'timestamp': self.timestamp.isoformat(),
            'total_chains': len(chains),
            'active_chains': sum(1 for c in chains.values()
                               if c['status'] == 'active'),
            'statistics': statistics,
            'column_info': {
                'total_tables_with_columns': total_column_sets,
                'average_columns_per_table': round(avg_columns_per_table, 2)
            }
        }
        return summary

    def save_chains_json(self, chains, filepath="chains.json"):
        """Save chains to JSON with proper type conversion"""

        def convert_to_native(obj):
            """Convert numpy types to native Python types"""
            if isinstance(obj, np.bool_):
                return bool(obj)
            elif isinstance(obj, (np.integer, np.int32, np.int64)):
                return int(obj)
            elif isinstance(obj, (np.floating, np.float32, np.float64)):
                return float(obj)
            elif isinstance(obj, np.ndarray):
                return obj.tolist()
            elif isinstance(obj, list):
                return [convert_to_native(item) for item in obj]
            elif isinstance(obj, dict):
                return {key: convert_to_native(value) for key, value in obj.items()}
            else:
                return obj

        # Convert all chains
        chains_export = convert_to_native(chains)

        with open(filepath, 'w', encoding='utf-8') as f:
            json.dump(chains_export, f, indent=2, ensure_ascii=False)
        return filepath

    def generate_html_report(self, chains, statistics):
        """Generate HTML report with column information"""
        html = f"""<!DOCTYPE html>
<html>
<head>
    <title>Chain Matching Report</title>
    <meta charset="utf-8">
    <style>
        body {{ font-family: Arial, sans-serif; margin: 20px; }}
        h1, h2, h3 {{ color: #333; }}
        .summary {{ background: #f0f0f0; padding: 15px; border-radius: 5px; margin: 20px 0; }}
        .chain {{ margin: 15px 0; padding: 10px; border-left: 3px solid #4CAF50; }}
        .active {{ background: #e8f5e9; }}
        .dormant {{ background: #fff3e0; border-color: #FF9800; }}
        .ended {{ background: #ffebee; border-color: #f44336; }}
        .table-info {{ margin-left: 20px; font-size: 0.9em; color: #666; }}
        .columns {{ font-size: 0.85em; color: #888; margin-left: 30px; }}
        .statistics {{ display: grid; grid-template-columns: repeat(auto-fit, minmax(200px, 1fr)); gap: 15px; }}
        .stat-card {{ background: white; padding: 10px; border: 1px solid #ddd; border-radius: 5px; }}
    </style>
</head>
<body>
    <h1>Table Chain Matching Report</h1>
    <p><strong>Generated:</strong> {self.timestamp.strftime('%Y-%m-%d %H:%M:%S')}</p>

    <div class="summary">
        <h2>Summary</h2>
        <div class="statistics">
            <div class="stat-card">
                <strong>Total Chains:</strong> {len(chains)}
            </div>
            <div class="stat-card">
                <strong>Active Chains:</strong> {sum(1 for c in chains.values() if c['status'] == 'active')}
            </div>
            <div class="stat-card">
                <strong>Dormant Chains:</strong> {sum(1 for c in chains.values() if c['status'] == 'dormant')}
            </div>
            <div class="stat-card">
                <strong>Ended Chains:</strong> {sum(1 for c in chains.values() if c['status'] == 'ended')}
            </div>
        </div>
    </div>

    <h2>Chain Details</h2>"""

        # Group chains by status
        active_chains = {k: v for k, v in chains.items() if v['status'] == 'active'}
        dormant_chains = {k: v for k, v in chains.items() if v['status'] == 'dormant'}
        ended_chains = {k: v for k, v in chains.items() if v['status'] == 'ended'}

        # Active Chains
        if active_chains:
            html += "<h3>Active Chains</h3>"
            for chain_id, chain in sorted(active_chains.items()):
                html += self._format_chain_html(chain_id, chain)

        # Dormant Chains
        if dormant_chains:
            html += "<h3>Dormant Chains</h3>"
            for chain_id, chain in sorted(dormant_chains.items()):
                html += self._format_chain_html(chain_id, chain)

        # Ended Chains (show only summary)
        if ended_chains:
            html += f"<h3>Ended Chains ({len(ended_chains)})</h3>"
            html += "<p>Chains that have not matched for multiple years and are considered ended.</p>"

        # Add statistics if available
        if statistics and 'year_by_year' in statistics:
            html += "<h2>Year-by-Year Statistics</h2>"
            html += "<table border='1' style='border-collapse: collapse; width: 100%;'>"
            html += "<tr><th>Year</th><th>Tables</th><th>Matches</th><th>Match Rate</th><th>Processing Time</th></tr>"

            for year, stats in sorted(statistics['year_by_year'].items()):
                html += f"""<tr>
                    <td>{year}</td>
                    <td>{stats.get('tables', 'N/A')}</td>
                    <td>{stats.get('matches', 'N/A')}</td>
                    <td>{stats.get('match_rate', 'N/A')}</td>
                    <td>{stats.get('processing_time', 'N/A')}</td>
                </tr>"""
            html += "</table>"

        html += """
</body>
</html>"""

        with open("report.html", "w", encoding='utf-8') as f:
            f.write(html)

        return "report.html"

    def _format_chain_html(self, chain_id, chain):
        """Format a single chain for HTML display"""
        status_class = chain['status']
        years_range = f"{min(chain['years'])}-{max(chain['years'])}" if chain['years'] else "N/A"

        html = f"""<div class="chain {status_class}">
            <strong>{chain_id}</strong>
            <span style="color: #666;">({len(chain['tables'])} tables, Years: {years_range})</span>
        """

        # Show first few tables with column info
        tables_to_show = min(3, len(chain['tables']))
        for i in range(tables_to_show):
            table = chain['tables'][i]
            year = chain['years'][i] if i < len(chain['years']) else 'N/A'
            header = chain['headers'][i] if i < len(chain['headers']) else 'No header'
            columns = chain.get('columns', [])[i] if i < len(chain.get('columns', [])) else []

            # Clean header for display
            clean_header = header.replace('\n', ' ')[:100]
            if len(header) > 100:
                clean_header += '...'

            html += f"""<div class="table-info">
                <strong>{table}</strong> ({year}): {clean_header}
            """

            # Show column information if available
            if columns:
                col_preview = ', '.join(columns[:5])
                if len(columns) > 5:
                    col_preview += f', ... ({len(columns)} total)'
                else:
                    col_preview += f' ({len(columns)} total)'
                html += f'<div class="columns">Columns: {col_preview}</div>'

            html += "</div>"

        if len(chain['tables']) > tables_to_show:
            html += f"<div class='table-info'>... and {len(chain['tables']) - tables_to_show} more tables</div>"

        # Show gaps if any
        if chain.get('gaps'):
            html += f"<div class='table-info' style='color: #FF9800;'>Gaps in years: {', '.join(map(str, chain['gaps']))}</div>"

        html += "</div>"
        return html

Writing report_gen.py


# Real Embeddings with Sentence Transformers

In [12]:
%%writefile real_embeddings.py
import numpy as np
import pickle
import os
import hashlib

try:
    from sentence_transformers import SentenceTransformer
    TRANSFORMER_AVAILABLE = True
except:
    TRANSFORMER_AVAILABLE = False
    print("Install with: !pip install sentence-transformers")

class RealEmbeddingGenerator:
    def __init__(self, model_name="sentence-transformers/LaBSE", cache_dir="cache"):
        self.cache_dir = cache_dir
        os.makedirs(cache_dir, exist_ok=True)
        self.embedding_cache = {}

        if TRANSFORMER_AVAILABLE:
            self.model = SentenceTransformer(model_name)
            self.dimension = self.model.get_sentence_embedding_dimension()
        else:
            self.model = None
            self.dimension = 768

    def get_text_hash(self, text):
        return hashlib.md5(text.encode('utf-8')).hexdigest()

    def generate_embedding(self, text, use_cache=True):
        text_hash = self.get_text_hash(text)

        if use_cache and text_hash in self.embedding_cache:
            return self.embedding_cache[text_hash]

        if self.model:
            embedding = self.model.encode(text, convert_to_numpy=True)
        else:
            # Fallback to deterministic random
            np.random.seed(int(text_hash[:8], 16) % 10000)
            embedding = np.random.randn(self.dimension)

        if use_cache:
            self.embedding_cache[text_hash] = embedding

        return embedding

    def generate_batch(self, texts, show_progress=True):
        if self.model:
            return self.model.encode(texts,
                                    batch_size=32,
                                    show_progress_bar=show_progress,
                                    convert_to_numpy=True)
        else:
            return np.array([self.generate_embedding(t) for t in texts])

    def save_cache(self):
        cache_file = os.path.join(self.cache_dir, "embedding_cache.pkl")
        with open(cache_file, 'wb') as f:
            pickle.dump(self.embedding_cache, f)

Writing real_embeddings.py


# Claude API Validation

In [13]:
%%writefile api_validator.py
import json
import time
import random
import os

class ClaudeAPIValidator:
    def __init__(self, api_key=None):
        self.api_key = api_key or os.getenv('CLAUDE_API_KEY')
        self.has_api = bool(self.api_key)
        self.validation_count = 0

    def validate_edge_case(self, chain_headers, table_header, similarity):
        """Validate uncertain match (0.85-0.97)"""
        self.validation_count += 1

        if self.has_api:
            return self._real_api_call(chain_headers, table_header, similarity)
        else:
            return self._mock_validation(similarity)

    def _mock_validation(self, similarity):
        """Mock validation for testing"""
        if similarity >= 0.92:
            return {'decision': 'accept', 'confidence': 0.9, 'reasoning': 'High similarity'}
        elif similarity >= 0.88:
            return {'decision': 'uncertain', 'confidence': 0.6, 'reasoning': 'Moderate similarity'}
        else:
            return {'decision': 'reject', 'confidence': 0.8, 'reasoning': 'Low similarity'}

    def _real_api_call(self, chain_headers, table_header, similarity):
        """Real API call (if implemented)"""
        # Placeholder for real Claude API implementation
        prompt = f"""
        Chain history: {chain_headers}
        New table: {table_header}
        Similarity: {similarity}
        Should these match?
        """

        # Would make actual API call here
        return self._mock_validation(similarity)

    def validate_conflict(self, table_header, competing_chains):
        """Resolve conflicts between multiple chains"""
        if self.has_api:
            # Real API logic
            pass
        else:
            # Mock: choose highest similarity
            best_chain = max(competing_chains, key=lambda x: x[1])
            return {
                'winning_chain': best_chain[0],
                'confidence': 0.8,
                'reasoning': 'Highest similarity score'
            }

    def validate_split(self, source_chain, target_tables):
        """Validate potential split"""
        if len(target_tables) >= 2:
            return {
                'decision': 'accept',
                'split_type': 'even_split' if len(target_tables) == 2 else 'fragmentation',
                'confidence': 0.7,
                'targets': [t[0] for t in target_tables[:3]]
            }
        return {'decision': 'reject', 'confidence': 0.9}

Writing api_validator.py


# Gap Handler with Reactivation

In [14]:
%%writefile gap_handler.py
import numpy as np

class GapHandler:
    def __init__(self, max_gap_years=3, reactivation_threshold=0.90):
        self.max_gap_years = max_gap_years
        self.reactivation_threshold = reactivation_threshold
        self.dormant_chains = {}
        self.ended_chains = {}

    def check_gaps(self, chains, current_year, matched_chains):
        """Check for gaps and handle dormant chains"""
        gap_report = {
            'new_dormant': [],
            'reactivated': [],
            'ended': [],
            'continuing_gaps': []
        }

        for chain_id, chain in chains.items():
            if chain['status'] == 'active' and chain_id not in matched_chains:
                # Chain has no match this year
                last_year = chain['years'][-1] if chain['years'] else 0
                gap_length = current_year - last_year

                if gap_length > self.max_gap_years:
                    # End chain
                    chain['status'] = 'ended'
                    self.ended_chains[chain_id] = chain
                    gap_report['ended'].append(chain_id)
                else:
                    # Mark dormant
                    chain['status'] = 'dormant'
                    chain['dormant_since'] = current_year
                    self.dormant_chains[chain_id] = chain
                    gap_report['new_dormant'].append(chain_id)

        return gap_report

    def check_reactivation(self, dormant_chain, new_tables, embeddings):
        """Check if dormant chain can be reactivated"""
        if dormant_chain['tables']:
            last_table = dormant_chain['tables'][-1]
            if last_table in embeddings:
                chain_emb = embeddings[last_table]

                candidates = []
                for table_id in new_tables:
                    if table_id in embeddings:
                        table_emb = embeddings[table_id]
                        similarity = self._compute_similarity(chain_emb, table_emb)

                        if similarity >= self.reactivation_threshold:
                            candidates.append((table_id, similarity))

                if candidates:
                    return max(candidates, key=lambda x: x[1])
        return None

    def _compute_similarity(self, emb1, emb2):
        """Compute cosine similarity"""
        from scipy.spatial.distance import cosine
        return (1 - cosine(emb1, emb2) + 1) / 2

Writing gap_handler.py


# Storage and Checkpointing

In [15]:
%%writefile storage_manager.py
import json
import pickle
import gzip
from datetime import datetime
from pathlib import Path

class StorageManager:
    def __init__(self, storage_dir="chain_storage"):
        self.storage_dir = Path(storage_dir)
        self.storage_dir.mkdir(parents=True, exist_ok=True)

        # Create subdirectories
        (self.storage_dir / "checkpoints").mkdir(exist_ok=True)
        (self.storage_dir / "backups").mkdir(exist_ok=True)
        (self.storage_dir / "embeddings").mkdir(exist_ok=True)

    def save_checkpoint(self, year, chains, statistics):
        """Save processing checkpoint"""
        checkpoint = {
            'year': year,
            'timestamp': datetime.now().isoformat(),
            'chains': chains,
            'statistics': statistics
        }

        filename = f"checkpoint_{year}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
        filepath = self.storage_dir / "checkpoints" / filename

        with gzip.open(filepath.with_suffix('.json.gz'), 'wt', encoding='utf-8') as f:
            json.dump(checkpoint, f, indent=2)

        return str(filepath)

    def load_checkpoint(self, year):
        """Load latest checkpoint for a year"""
        checkpoint_dir = self.storage_dir / "checkpoints"
        pattern = f"checkpoint_{year}_*.json.gz"

        files = list(checkpoint_dir.glob(pattern))
        if files:
            latest = max(files, key=lambda f: f.stat().st_mtime)
            with gzip.open(latest, 'rt', encoding='utf-8') as f:
                return json.load(f)
        return None

    def save_embeddings(self, embeddings, year):
        """Save embeddings for a year"""
        filepath = self.storage_dir / "embeddings" / f"embeddings_{year}.pkl"
        with open(filepath, 'wb') as f:
            pickle.dump(embeddings, f)

    def load_embeddings(self, year):
        """Load embeddings for a year"""
        filepath = self.storage_dir / "embeddings" / f"embeddings_{year}.pkl"
        if filepath.exists():
            with open(filepath, 'rb') as f:
                return pickle.load(f)
        return None

    def backup_chains(self, chains):
        """Create backup of chains"""
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        backup_file = self.storage_dir / "backups" / f"chains_backup_{timestamp}.json.gz"

        with gzip.open(backup_file, 'wt', encoding='utf-8') as f:
            json.dump(chains, f, indent=2)

Writing storage_manager.py


# Comprehensive Statistics Tracker

In [16]:
%%writefile statistics_tracker.py
import numpy as np
from collections import defaultdict
import json
import datetime
from datetime import datetime

class StatisticsTracker:
    def __init__(self):
        self.match_history = []
        self.year_statistics = {}
        self.chain_statistics = defaultdict(lambda: {
            'length': 0,
            'gaps': [],
            'similarity_scores': [],
            'api_validations': 0
        })

        self.global_stats = {
            'total_years_processed': 0,
            'total_matches': 0,
            'total_chains': 0,
            'total_api_calls': 0,
            'total_splits': 0,
            'total_merges': 0
        }

        self.similarity_distributions = defaultdict(list)

    def record_match(self, chain_id, table_id, year, similarity, match_type='confident'):
        """Record a single match"""
        self.match_history.append({
            'chain': chain_id,
            'table': table_id,
            'year': year,
            'similarity': similarity,
            'type': match_type,
            'timestamp': str(datetime.now())
        })

        self.chain_statistics[chain_id]['length'] += 1
        self.chain_statistics[chain_id]['similarity_scores'].append(similarity)
        self.similarity_distributions[year].append(similarity)
        self.global_stats['total_matches'] += 1

    def record_year(self, year, tables_count, matches_count,
                    unmatched_tables, unmatched_chains, processing_time):
        """Record year statistics"""
        self.year_statistics[year] = {
            'tables': tables_count,
            'matches': matches_count,
            'unmatched_tables': len(unmatched_tables),
            'unmatched_chains': len(unmatched_chains),
            'match_rate': matches_count / tables_count if tables_count > 0 else 0,
            'processing_time': processing_time,
            'similarity_distribution': {}
        }

        if year in self.similarity_distributions:
            scores = self.similarity_distributions[year]
            self.year_statistics[year]['similarity_distribution'] = {
                'mean': float(np.mean(scores)),
                'median': float(np.median(scores)),
                'std': float(np.std(scores)),
                'min': float(np.min(scores)),
                'max': float(np.max(scores))
            }

        self.global_stats['total_years_processed'] += 1

    def get_summary(self):
        """Get comprehensive summary"""
        chain_lengths = [s['length'] for s in self.chain_statistics.values()]

        return {
            'overview': {
                'total_years': self.global_stats['total_years_processed'],
                'total_matches': self.global_stats['total_matches'],
                'total_chains': len(self.chain_statistics),
                'match_rate': f"{np.mean([y['match_rate'] for y in self.year_statistics.values()])*100:.1f}%" if self.year_statistics else "0%"
            },
            'chain_statistics': {
                'average_length': np.mean(chain_lengths) if chain_lengths else 0,
                'max_length': max(chain_lengths) if chain_lengths else 0,
                'min_length': min(chain_lengths) if chain_lengths else 0,
                'chains_with_gaps': sum(1 for c in self.chain_statistics.values() if c['gaps'])
            },
            'year_by_year': {
                year: {
                    'tables': stats['tables'],
                    'matches': stats['matches'],
                    'match_rate': f"{stats['match_rate']*100:.1f}%",
                    'processing_time': f"{stats['processing_time']:.2f}s"
                }
                for year, stats in self.year_statistics.items()
            }
        }

Writing statistics_tracker.py


# Visualization Generator

In [17]:
%%writefile visualization.py
import json
import os
from datetime import datetime
import numpy as np
try:
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    PLOTLY_AVAILABLE = True
except:
    PLOTLY_AVAILABLE = False


class VisualizationGenerator:
    def __init__(self):
        self.colors = ['#4CAF50', '#FF9800', '#9C27B0', '#F44336', '#2196F3']

    def create_sankey(self, chains, sim_matrix_data=None):
        """Create enhanced Sankey diagram with similarity scores and correlation matrix"""
        if not PLOTLY_AVAILABLE:
            print("Install plotly: !pip install plotly")
            return None

        # Create subplots - Sankey on top, heatmap below
        fig = make_subplots(
            rows=2, cols=1,
            row_heights=[0.7, 0.3],
            specs=[[{"type": "sankey"}],
                   [{"type": "heatmap"}]],
            subplot_titles=("Table Chain Evolution", "Similarity Matrix")
        )

        # Build Sankey data
        nodes = []
        node_labels = []
        sources = []
        targets = []
        values = []
        link_labels = []
        link_colors = []

        node_map = {}
        header_map = {}
        columns_map = {}  # New: track columns for hover text
        node_idx = 0

        for chain_id, chain in chains.items():
            for i, table in enumerate(chain['tables']):
                if table not in node_map:
                    node_map[table] = node_idx
                    header = chain['headers'][i] if i < len(chain['headers']) else 'No header'
                    header_map[table] = header

                    # Get column information
                    columns = chain.get('columns', [])[i] if i < len(chain.get('columns', [])) else []
                    columns_map[table] = columns

                    clean_header = header.replace('\n', ' ')[:50] + '...' if len(header) > 50 else header.replace('\n', ' ')

                    # Enhanced node label with column count
                    col_info = f"<br>Columns: {len(columns)}" if columns else ""
                    node_labels.append(f"{table}<br>Year: {chain['years'][i]}<br>{clean_header}{col_info}")
                    node_idx += 1

                if i > 0:
                    prev_table = chain['tables'][i-1]
                    sources.append(node_map[prev_table])
                    targets.append(node_map[table])
                    values.append(1)

                    # Get similarity and API info if available
                    similarity = chain.get('similarities', [])[i-1] if i-1 < len(chain.get('similarities', [])) else 0.95
                    api_used = chain.get('api_validated', [])[i-1] if i-1 < len(chain.get('api_validated', [])) else False

                    # Create detailed hover text with column preview
                    source_header = header_map.get(prev_table, 'No header')
                    target_header = header_map.get(table, 'No header')
                    api_text = "✓ API Validated" if api_used else "Auto-matched"

                    # Add column information if available
                    source_cols = columns_map.get(prev_table, [])
                    target_cols = columns_map.get(table, [])

                    col_preview_source = ""
                    col_preview_target = ""

                    if source_cols:
                        col_preview_source = f"<br>Columns ({len(source_cols)}): {', '.join(source_cols[:3])}"
                        if len(source_cols) > 3:
                            col_preview_source += "..."

                    if target_cols:
                        col_preview_target = f"<br>Columns ({len(target_cols)}): {', '.join(target_cols[:3])}"
                        if len(target_cols) > 3:
                            col_preview_target += "..."

                    hover_text = (f"<b>Similarity: {similarity:.3f}</b><br>"
                                f"{api_text}<br><br>"
                                f"<b>Source:</b> {prev_table}<br>{source_header}{col_preview_source}<br><br>"
                                f"<b>Target:</b> {table}<br>{target_header}{col_preview_target}")
                    link_labels.append(hover_text)

                    # Color based on similarity
                    if similarity >= 0.97:
                        color = 'rgba(76, 175, 80, 0.5)'  # Green
                    elif similarity >= 0.90:
                        color = 'rgba(255, 193, 7, 0.5)'  # Amber
                    elif similarity >= 0.85:
                        color = 'rgba(255, 152, 0, 0.5)'  # Orange
                    else:
                        color = 'rgba(244, 67, 54, 0.5)'  # Red

                    if api_used:
                        color = color.replace('0.5', '0.8')  # Darker if API validated

                    link_colors.append(color)

        # Add Sankey to subplot
        sankey = go.Sankey(
            node=dict(
                pad=15,
                thickness=20,
                line=dict(color="black", width=0.5),
                label=node_labels,
                hovertemplate='%{label}<extra></extra>'
            ),
            link=dict(
                source=sources,
                target=targets,
                value=values,
                label=link_labels,
                color=link_colors,
                hovertemplate='%{label}<extra></extra>'
            )
        )

        fig.add_trace(sankey, row=1, col=1)

        # Add similarity matrix heatmap if available
        if sim_matrix_data and 'matrix' in sim_matrix_data:
            matrix = sim_matrix_data['matrix']
            chain_ids = [c.split('_')[-1] if c.startswith('chain_') else c
                        for c in sim_matrix_data.get('chain_ids', [])]
            table_ids = sim_matrix_data.get('table_ids', [])

            heatmap = go.Heatmap(
                z=matrix,
                x=table_ids,
                y=chain_ids,
                colorscale='RdYlGn',
                zmin=0,
                zmax=1,
                text=np.round(matrix, 3),
                texttemplate='%{text}',
                textfont={"size": 8},
                hovertemplate='Chain: %{y}<br>Table: %{x}<br>Similarity: %{z:.3f}<extra></extra>',
                colorbar=dict(title="Similarity", len=0.3, y=0.15)
            )

            fig.add_trace(heatmap, row=2, col=1)

        # Update layout
        fig.update_layout(
            title_text="Table Chain Analysis with Similarity Metrics and Column Information",
            height=1000,
            width=1400,
            showlegend=False,
            font_size=10
        )

        return fig

    def save_graph_json(self, chains, filepath="graph.json"):
        """Save graph structure as JSON with similarity scores and column information"""
        graph = {
            'nodes': [],
            'edges': [],
            'metadata': {
                'created': str(datetime.now()),
                'total_chains': len(chains),
                'active_chains': sum(1 for c in chains.values() if c['status'] == 'active')
            }
        }

        for chain_id, chain in chains.items():
            for i, table in enumerate(chain['tables']):
                # Include column information in node data
                columns = chain.get('columns', [])[i] if i < len(chain.get('columns', [])) else []

                graph['nodes'].append({
                    'id': table,
                    'chain': chain_id,
                    'year': chain['years'][i] if i < len(chain['years']) else 0,
                    'header': chain['headers'][i] if i < len(chain['headers']) else '',
                    'columns': columns,  # New: include column information
                    'column_count': len(columns)  # New: include column count for easy reference
                })

                if i > 0:
                    similarity = chain.get('similarities', [])[i-1] if i-1 < len(chain.get('similarities', [])) else None
                    api_used = chain.get('api_validated', [])[i-1] if i-1 < len(chain.get('api_validated', [])) else False

                    graph['edges'].append({
                        'source': chain['tables'][i-1],
                        'target': table,
                        'type': 'continuation',
                        'similarity': similarity,
                        'api_validated': api_used
                    })

        with open(filepath, 'w', encoding='utf-8') as f:
            json.dump(graph, f, indent=2, ensure_ascii=False)

        return filepath

Writing visualization.py


# Complex N:N Relationships

In [18]:
%%writefile complex_relationships.py
import numpy as np
from enum import Enum
from datetime import datetime

class RelationshipType(Enum):
    ONE_TO_ONE = "1:1"
    ONE_TO_MANY = "1:N"
    MANY_TO_ONE = "N:1"
    MANY_TO_MANY = "N:N"

class ComplexRelationshipDetector:
    def __init__(self):
        self.complex_relationships = []

    def detect_complex(self, sim_matrix, splits, merges):
        """Detect N:N complex reorganizations"""
        split_tables = set()
        for split in splits:
            split_tables.update([t[0] for t in split['targets']])

        merge_chains = set()
        for merge in merges:
            merge_chains.update([c[0] for c in merge['sources']])

        # Find overlapping splits and merges (N:N)
        for split in splits:
            if split['chain'] in merge_chains:
                for merge in merges:
                    if split['chain'] in [c[0] for c in merge['sources']]:
                        self.complex_relationships.append({
                            'type': RelationshipType.MANY_TO_MANY,
                            'chains': list(set([split['chain']] + [c[0] for c in merge['sources']])),
                            'tables': list(set([merge['table']] + [t[0] for t in split['targets']])),
                            'confidence': 0.7
                        })

        return self.complex_relationships

Writing complex_relationships.py


#

# NetworkX Graph Builder

In [19]:
%%writefile networkx_builder.py
try:
    import networkx as nx
    NX_AVAILABLE = True
except:
    NX_AVAILABLE = False

class NetworkXGraphBuilder:
    def __init__(self):
        self.G = None if not NX_AVAILABLE else nx.DiGraph()

    def build_graph(self, chains):
        """Build complete NetworkX graph"""
        if not NX_AVAILABLE:
            return None

        self.G = nx.DiGraph()

        # Add all nodes
        for chain_id, chain in chains.items():
            for i, table in enumerate(chain['tables']):
                self.G.add_node(table,
                              chain=chain_id,
                              year=chain['years'][i] if i < len(chain['years']) else 0,
                              header=chain['headers'][i] if i < len(chain['headers']) else '')

        # Add edges
        for chain_id, chain in chains.items():
            for i in range(1, len(chain['tables'])):
                self.G.add_edge(chain['tables'][i-1],
                              chain['tables'][i],
                              weight=1.0,
                              type='continuation')

        return self.G

    def analyze_graph(self):
        """Analyze graph properties"""
        if not self.G:
            return {}

        return {
            'nodes': self.G.number_of_nodes(),
            'edges': self.G.number_of_edges(),
            'connected_components': nx.number_weakly_connected_components(self.G),
            'average_degree': sum(dict(self.G.degree()).values()) / self.G.number_of_nodes()
        }

Writing networkx_builder.py


# Full Conflict Resolution

In [20]:
%%writefile conflict_resolver.py
class ConflictResolver:
    def __init__(self):
        self.conflicts = {}
        self.resolutions = {}

    def detect_conflicts(self, sim_matrix, threshold=0.85):
        """Detect all conflicts"""
        matrix = sim_matrix['matrix']
        chain_ids = sim_matrix['chain_ids']
        table_ids = sim_matrix['table_ids']

        for j, table_id in enumerate(table_ids):
            claimants = []
            for i, chain_id in enumerate(chain_ids):
                if matrix[i, j] >= threshold:
                    claimants.append((chain_id, matrix[i, j]))

            if len(claimants) > 1:
                self.conflicts[table_id] = {
                    'claimants': claimants,
                    'max_similarity': max(c[1] for c in claimants)
                }

        return self.conflicts

    def resolve_conflicts(self, conflicts, api_validator=None):
        """Resolve all conflicts"""
        for table_id, conflict in conflicts.items():
            if api_validator:
                resolution = api_validator.validate_conflict(
                    table_id, conflict['claimants']
                )
                self.resolutions[table_id] = resolution
            else:
                # Default: highest similarity wins
                winner = max(conflict['claimants'], key=lambda x: x[1])
                self.resolutions[table_id] = {
                    'winning_chain': winner[0],
                    'confidence': winner[1]
                }

        return self.resolutions

Writing conflict_resolver.py


# API Response Handler

In [21]:
%%writefile response_handler.py
from enum import Enum

class DecisionAction(Enum):
    CONFIRM = "confirm"
    REJECT = "reject"
    SPLIT = "split"
    MERGE = "merge"
    MANUAL = "manual"

class APIResponseHandler:
    def __init__(self):
        self.decisions = []
        self.manual_queue = []

    def process_response(self, api_response, match_type):
        """Process API validation response"""
        decision = api_response.get('decision', 'uncertain')
        confidence = api_response.get('confidence', 0.5)

        if decision == 'accept' and confidence >= 0.7:
            action = DecisionAction.CONFIRM
        elif decision == 'reject' and confidence >= 0.7:
            action = DecisionAction.REJECT
        else:
            action = DecisionAction.MANUAL
            self.manual_queue.append(api_response)

        self.decisions.append({
            'action': action,
            'confidence': confidence,
            'type': match_type,
            'response': api_response
        })

        return action

Writing response_handler.py


# Parameter Tuning Suite

In [22]:
%%writefile parameter_tuner.py
import json
import numpy as np

class ParameterTuner:
    def __init__(self):
        self.param_history = []
        self.optimal_params = None

    def grid_search(self, param_ranges, validation_data):
        """Grid search for optimal parameters"""
        best_score = 0
        best_params = {}

        # Example grid search
        for sim_thresh in param_ranges.get('similarity_threshold', [0.85]):
            for split_thresh in param_ranges.get('split_threshold', [0.80]):
                score = self._evaluate_params({
                    'similarity_threshold': sim_thresh,
                    'split_threshold': split_thresh
                }, validation_data)

                if score > best_score:
                    best_score = score
                    best_params = {
                        'similarity_threshold': sim_thresh,
                        'split_threshold': split_thresh
                    }

        self.optimal_params = best_params
        return best_params

    def _evaluate_params(self, params, validation_data):
        """Evaluate parameter set"""
        # Mock evaluation - in reality would run matching and compare
        return np.random.random()

    def suggest_adjustments(self, current_stats):
        """Suggest parameter adjustments based on statistics"""
        suggestions = []

        if current_stats.get('match_rate', 0) < 0.7:
            suggestions.append("Consider lowering similarity_threshold")

        if current_stats.get('false_positives', 0) > 0.1:
            suggestions.append("Consider raising similarity_threshold")

        return suggestions

Writing parameter_tuner.py


# Complete Testing Suite

In [23]:
%%writefile test_suite.py
import unittest
import numpy as np

class TestCompleteSystem(unittest.TestCase):
    def test_similarity_computation(self):
        """Test similarity computation"""
        emb1 = np.array([1, 0, 0])
        emb2 = np.array([1, 0, 0])
        emb3 = np.array([0, 1, 0])

        from scipy.spatial.distance import cosine
        sim12 = 1 - cosine(emb1, emb2)
        sim13 = 1 - cosine(emb1, emb3)

        self.assertAlmostEqual(sim12, 1.0)
        self.assertAlmostEqual(sim13, 0.0)

    def test_hungarian_matching(self):
        """Test Hungarian algorithm"""
        from scipy.optimize import linear_sum_assignment

        cost = np.array([[1, 2], [3, 4]])
        row_ind, col_ind = linear_sum_assignment(cost)

        self.assertEqual(len(row_ind), 2)
        self.assertEqual(len(col_ind), 2)

    def test_conflict_detection(self):
        """Test conflict detection"""
        matrix = np.array([[0.9, 0.3], [0.88, 0.4]])

        conflicts = []
        for j in range(matrix.shape[1]):
            high_sim = []
            for i in range(matrix.shape[0]):
                if matrix[i, j] >= 0.85:
                    high_sim.append(i)
            if len(high_sim) > 1:
                conflicts.append(j)

        self.assertEqual(len(conflicts), 0)  # No conflicts in this example

def run_all_tests():
    """Run complete test suite"""
    loader = unittest.TestLoader()
    suite = loader.loadTestsFromTestCase(TestCompleteSystem)
    runner = unittest.TextTestRunner(verbosity=2)
    result = runner.run(suite)
    return result.wasSuccessful()

Writing test_suite.py


# Final Complete Orchestrator

In [ ]:
%%writefile final_complete_processor.py
import time
import os
from datetime import datetime
from scipy.spatial.distance import cosine

# Import ALL components
from config import MatchingConfig
from hebrew_processor import HebrewProcessor
from table_loader import TableLoader
from real_embeddings import RealEmbeddingGenerator
from similarity import SimilarityBuilder
from hungarian import HungarianMatcher
from split_merge import SplitMergeDetector
from complex_relationships import ComplexRelationshipDetector
from chains import ChainManager
from api_validator import ClaudeAPIValidator
from gap_handler import GapHandler
from storage_manager import StorageManager
from statistics_tracker import StatisticsTracker
from visualization import VisualizationGenerator
from report_gen import ReportGenerator
from networkx_builder import NetworkXGraphBuilder
from conflict_resolver import ConflictResolver
from response_handler import APIResponseHandler
from parameter_tuner import ParameterTuner
from test_suite import run_all_tests

def process_table_chains_final_complete():
    """Complete processing with chapter-by-chapter matching and proper validation"""
    print("="*60)
    print("CHAPTER-BY-CHAPTER TABLE CHAIN MATCHING SYSTEM")
    print("="*60)

    # Run tests first
    print("\nRunning system tests...")
    tests_passed = run_all_tests()
    print(f"Tests: {'PASSED' if tests_passed else 'FAILED'}")

    start_time = time.time()

    # Initialize components
    config = MatchingConfig()
    config.use_api_validation = True  # Enable API validation
    hebrew_proc = HebrewProcessor()

    # Initialize loader
    loader = TableLoader(
        tables_dir=config.tables_dir,
        reference_json=config.reference_json,
        columns_json=config.columns_json
    )

    embedder = RealEmbeddingGenerator()
    sim_builder = SimilarityBuilder()
    matcher = HungarianMatcher(config.similarity_threshold)
    split_detector = SplitMergeDetector()
    complex_detector = ComplexRelationshipDetector()
    api_validator = ClaudeAPIValidator(config.api_key)
    gap_handler = GapHandler(config.max_gap_years)
    storage_mgr = StorageManager()
    stats_tracker = StatisticsTracker()
    visualizer = VisualizationGenerator()
    reporter = ReportGenerator()
    nx_builder = NetworkXGraphBuilder()
    conflict_resolver = ConflictResolver()
    response_handler = APIResponseHandler()
    param_tuner = ParameterTuner()

    # Load tables
    print("\n1. Loading tables...")
    n_tables = loader.load_metadata()
    print(f"   Loaded {n_tables} tables")

    # Check if reference files exist
    if not os.path.exists(config.reference_json):
        print(f"   Warning: {config.reference_json} not found!")
        return None, None

    if not os.path.exists(config.columns_json):
        print(f"   Note: {config.columns_json} not found, proceeding without column data")

    # Reorganize tables by chapter and year
    tables_by_chapter_year = {}
    for tid, metadata in loader.tables_metadata.items():
        chapter = metadata['chapter']
        year = metadata['year']

        if chapter not in tables_by_chapter_year:
            tables_by_chapter_year[chapter] = {}
        if year not in tables_by_chapter_year[chapter]:
            tables_by_chapter_year[chapter][year] = []
        tables_by_chapter_year[chapter][year].append(tid)

    # TEST MODE: Only process Chapter 1
    # Change to list(sorted(tables_by_chapter_year.keys())) for all chapters
    chapters_to_process = [1]

    print(f"\n2. Chapter Organization:")
    print(f"   Found {len(tables_by_chapter_year)} chapters total")
    print(f"   Processing chapters: {chapters_to_process}")

    all_chapter_chains = {}
    all_chapter_stats = {}

    # Process each chapter independently
    for chapter in chapters_to_process:
        print(f"\n{'='*60}")
        print(f"PROCESSING CHAPTER {chapter}")
        print(f"{'='*60}")

        if chapter not in tables_by_chapter_year:
            print(f"   No tables found for chapter {chapter}")
            continue

        chapter_years = sorted(tables_by_chapter_year[chapter].keys())
        print(f"   Years available: {min(chapter_years)} to {max(chapter_years)}")

        # Initialize fresh components for this chapter
        chain_mgr = ChainManager()
        chapter_stats = StatisticsTracker()

        # Initialize chains with first year
        first_year = chapter_years[0]
        first_year_tables = {tid: loader.tables_metadata[tid]
                            for tid in tables_by_chapter_year[chapter][first_year]}

        chain_mgr.initialize_from_first_year(first_year_tables)
        print(f"   Initialized {len(chain_mgr.chains)} chains for Year {first_year}")

        # Generate embeddings for all tables in this chapter
        print(f"\n   Generating embeddings for chapter {chapter}...")
        chapter_embeddings = {}

        for year in chapter_years:
            year_count = 0
            for tid in tables_by_chapter_year[chapter][year]:
                text = hebrew_proc.process_header(loader.tables_metadata[tid]['header'])
                embedding = embedder.generate_embedding(text)
                chapter_embeddings[tid] = embedding
                year_count += 1
            print(f"      Year {year}: {year_count} embeddings")

        last_sim_matrix = None

        # Process each subsequent year for this chapter
        for year in chapter_years[1:]:
            print(f"\n   Processing Chapter {chapter}, Year {year}...")
            year_start = time.time()

            # Get embeddings for matching
            chain_embeddings = chain_mgr.get_chain_embeddings(chapter_embeddings)
            table_embeddings = {tid: chapter_embeddings[tid]
                              for tid in tables_by_chapter_year[chapter][year]
                              if tid in chapter_embeddings}

            print(f"      Active chains: {len(chain_embeddings)}, Tables: {len(table_embeddings)}")

            if not table_embeddings:
                print(f"      No tables to match for year {year}")
                continue

            # Build similarity matrix
            sim_matrix = sim_builder.compute_similarity_matrix(
                chain_embeddings, table_embeddings
            )
            last_sim_matrix = sim_matrix

            # Detect conflicts
            conflicts = conflict_resolver.detect_conflicts(sim_matrix)
            if conflicts:
                print(f"      Conflicts detected: {len(conflicts)}")
                resolutions = conflict_resolver.resolve_conflicts(conflicts, api_validator)

            # Hungarian matching
            matching_result = matcher.find_optimal_matching(sim_matrix)
            print(f"      Initial matches found: {len(matching_result['matches'])}")

            # Detect splits and merges
            splits = split_detector.detect_splits(sim_matrix)
            merges = split_detector.detect_merges(sim_matrix)
            complex_rels = complex_detector.detect_complex(sim_matrix, splits, merges)

            if splits:
                print(f"      Splits detected: {len(splits)}")
            if merges:
                print(f"      Merges detected: {len(merges)}")
            if complex_rels:
                print(f"      Complex N:N relationships: {len(complex_rels)}")

            # FIX 1: Proper API validation with rejection of low-confidence matches
            validated_matches = []
            for chain_id, table_id, similarity in matching_result['matches']:
                # Reject low confidence matches unless API confirms
                if similarity < 0.97:
                    if config.use_api_validation and 0.85 <= similarity:
                        validation = api_validator.validate_edge_case(
                            chain_mgr.chains[chain_id]['headers'],
                            loader.tables_metadata[table_id]['header'],
                            similarity
                        )
                        action = response_handler.process_response(validation, 'edge_case')
                        if action.value == 'confirm':
                            validated_matches.append({
                                'chain_id': chain_id,
                                'table_id': table_id,
                                'similarity': similarity,
                                'api_validated': True
                            })
                            print(f"      API confirmed: {chain_id} -> {table_id} (sim={similarity:.3f})")
                        else:
                            print(f"      API rejected: {chain_id} -> {table_id} (sim={similarity:.3f})")
                            # Mark table as unmatched since it was rejected
                            if table_id not in matching_result['unmatched_tables']:
                                matching_result['unmatched_tables'].append(table_id)
                    else:
                        print(f"      Rejected low confidence: {chain_id} -> {table_id} (sim={similarity:.3f})")
                        # Mark table as unmatched
                        if table_id not in matching_result['unmatched_tables']:
                            matching_result['unmatched_tables'].append(table_id)
                else:
                    # High confidence match, accept
                    validated_matches.append({
                        'chain_id': chain_id,
                        'table_id': table_id,
                        'similarity': similarity,
                        'api_validated': False
                    })

            print(f"      Validated matches: {len(validated_matches)}")

            # Update chains
            chain_mgr.update_chains(validated_matches, year, loader.tables_metadata)

            # Handle gaps
            matched_chains = {m['chain_id'] for m in validated_matches}
            gap_report = gap_handler.check_gaps(chain_mgr.chains, year, matched_chains)

            if gap_report['new_dormant']:
                print(f"      New dormant chains: {len(gap_report['new_dormant'])}")
            if gap_report['ended']:
                print(f"      Ended chains: {len(gap_report['ended'])}")

            # FIX 2: Try to reactivate dormant chains
            reactivated_count = 0
            for chain_id, chain in chain_mgr.chains.items():
                if chain['status'] == 'dormant' and chain_id not in matched_chains:
                    # Try matching this dormant chain to unmatched tables
                    if chain['tables']:
                        last_table = chain['tables'][-1]
                        if last_table in chapter_embeddings:
                            chain_emb = chapter_embeddings[last_table]
                            for table_id in list(matching_result['unmatched_tables']):  # Use list() to avoid modification during iteration
                                if table_id in chapter_embeddings:
                                    table_emb = chapter_embeddings[table_id]
                                    similarity = (1 - cosine(chain_emb, table_emb) + 1) / 2

                                    # Check if high confidence or API validates
                                    should_reactivate = False
                                    if similarity >= 0.97:
                                        should_reactivate = True
                                    elif config.use_api_validation and 0.85 <= similarity:
                                        validation = api_validator.validate_edge_case(
                                            chain['headers'],
                                            loader.tables_metadata[table_id]['header'],
                                            similarity
                                        )
                                        action = response_handler.process_response(validation, 'edge_case')
                                        if action.value == 'confirm':
                                            should_reactivate = True

                                    if should_reactivate:
                                        chain['status'] = 'active'
                                        chain['tables'].append(table_id)
                                        chain['years'].append(year)
                                        chain['headers'].append(loader.tables_metadata[table_id]['header'])
                                        chain['columns'].append(loader.tables_metadata[table_id].get('columns', []))
                                        chain['similarities'].append(similarity)
                                        chain['api_validated'].append(similarity < 0.97)
                                        matching_result['unmatched_tables'].remove(table_id)
                                        reactivated_count += 1
                                        print(f"      Reactivated chain {chain_id} with {table_id} (sim={similarity:.3f})")
                                        break

            if reactivated_count > 0:
                print(f"      Total chains reactivated: {reactivated_count}")

            # FIX 3: Create new chains for unmatched tables
            new_chains_count = 0
            for table_id in matching_result['unmatched_tables']:
                if table_id in loader.tables_metadata:
                    new_chain_id = f"chain_{table_id}"
                    chain_mgr.chains[new_chain_id] = {
                        'id': new_chain_id,
                        'tables': [table_id],
                        'years': [year],
                        'headers': [loader.tables_metadata[table_id]['header']],
                        'columns': [loader.tables_metadata[table_id].get('columns', [])],
                        'status': 'active',
                        'gaps': [],
                        'similarities': [],
                        'api_validated': []
                    }
                    new_chains_count += 1

            if new_chains_count > 0:
                print(f"      Created {new_chains_count} new chains for unmatched tables")

            # Record statistics for this chapter
            for match in validated_matches:
                chapter_stats.record_match(
                    match['chain_id'],
                    match['table_id'],
                    year,
                    match['similarity'],
                    'confident' if match['similarity'] >= 0.97 else 'uncertain'
                )

            year_time = time.time() - year_start
            chapter_stats.record_year(
                year, len(tables_by_chapter_year[chapter][year]),
                len(validated_matches),
                matching_result['unmatched_tables'],
                matching_result['unmatched_chains'],
                year_time
            )

        # Store results for this chapter
        all_chapter_chains[chapter] = chain_mgr.chains
        all_chapter_stats[chapter] = chapter_stats.get_summary()

        # Generate outputs for this chapter
        print(f"\n   Generating outputs for Chapter {chapter}...")

        # Create chapter-specific output files
        chapter_dir = f"output_chapter_{chapter}"
        os.makedirs(chapter_dir, exist_ok=True)

        # Visualizations
        sankey = visualizer.create_sankey(chain_mgr.chains, last_sim_matrix)
        if sankey:
            sankey_file = f"{chapter_dir}/sankey_chapter_{chapter}.html"
            sankey.write_html(sankey_file)
            print(f"      Created {sankey_file}")

        # Reports
        chains_file = f"{chapter_dir}/chains_chapter_{chapter}.json"
        reporter.save_chains_json(chain_mgr.chains, chains_file)
        print(f"      Created {chains_file}")

        html_file = f"{chapter_dir}/report_chapter_{chapter}.html"
        with open(html_file, 'w', encoding='utf-8') as f:
            html = f"""<!DOCTYPE html>
<html>
<head>
    <title>Chapter {chapter} Chain Report</title>
    <meta charset="utf-8">
    <style>
        body {{ font-family: Arial, sans-serif; margin: 20px; }}
        .summary {{ background: #f0f0f0; padding: 15px; margin: 20px 0; }}
    </style>
</head>
<body>
    <h1>Chapter {chapter} Chain Matching Report</h1>
    <div class="summary">
        <h2>Summary</h2>
        <p>Total chains: {len(chain_mgr.chains)}</p>
        <p>Active chains: {sum(1 for c in chain_mgr.chains.values() if c['status'] == 'active')}</p>
        <p>Dormant chains: {sum(1 for c in chain_mgr.chains.values() if c['status'] == 'dormant')}</p>
        <p>Year range: {min(chapter_years)} - {max(chapter_years)}</p>
    </div>
</body>
</html>"""
            f.write(html)
        print(f"      Created {html_file}")

        # Chapter summary
        print(f"\n   Chapter {chapter} Summary:")
        print(f"      Total chains: {len(chain_mgr.chains)}")
        print(f"      Active chains: {sum(1 for c in chain_mgr.chains.values() if c['status'] == 'active')}")
        print(f"      Dormant chains: {sum(1 for c in chain_mgr.chains.values() if c['status'] == 'dormant')}")
        print(f"      Total matches: {chapter_stats.global_stats['total_matches']}")

    # Final summary
    total_time = time.time() - start_time
    print(f"\n{'='*60}")
    print(f"✅ COMPLETE Processing finished in {total_time:.2f} seconds")
    print(f"   Chapters processed: {len(all_chapter_chains)}")

    for chapter in chapters_to_process:
        if chapter in all_chapter_chains:
            print(f"\n   Chapter {chapter}:")
            print(f"      Chains: {len(all_chapter_chains[chapter])}")
            print(f"      Active: {sum(1 for c in all_chapter_chains[chapter].values() if c['status'] == 'active')}")
            print(f"      Dormant: {sum(1 for c in all_chapter_chains[chapter].values() if c['status'] == 'dormant')}")

    if config.use_api_validation:
        print(f"\n   Total API validations: {api_validator.validation_count}")

    return all_chapter_chains, all_chapter_stats

if __name__ == "__main__":
    chains, statistics = process_table_chains_final_complete()

# To process ALL chapters instead of just Chapter 1, change line 87:
chapters_to_process = list(sorted(tables_by_chapter_year.keys()))

# Final Execution Window

In [25]:
# Install ALL required packages
!pip install scipy sentence-transformers plotly networkx -q

In [26]:
!rm -rf chain_storage/embeddings/*

In [27]:
import os
import importlib
import report_gen
importlib.reload(report_gen)

os.environ['CLAUDE_API_KEY'] = "put key here"

from final_complete_processor import process_table_chains_final_complete
chains, statistics = process_table_chains_final_complete()

test_conflict_detection (test_suite.TestCompleteSystem.test_conflict_detection)
Test conflict detection ... FAIL
test_hungarian_matching (test_suite.TestCompleteSystem.test_hungarian_matching)
Test Hungarian algorithm ... ok
test_similarity_computation (test_suite.TestCompleteSystem.test_similarity_computation)
Test similarity computation ... ok

FAIL: test_conflict_detection (test_suite.TestCompleteSystem.test_conflict_detection)
Test conflict detection
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/content/test_suite.py", line 41, in test_conflict_detection
    self.assertEqual(len(conflicts), 0)  # No conflicts in this example
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError: 1 != 0

----------------------------------------------------------------------
Ran 3 tests in 0.007s

FAILED (failures=1)


CHAPTER-BY-CHAPTER TABLE CHAIN MATCHING SYSTEM

Running system tests...
Tests: FAILED


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

2_Dense/model.safetensors:   0%|          | 0.00/2.36M [00:00<?, ?B/s]


1. Loading tables...
   Loaded 5207 tables

2. Chapter Organization:
   Found 15 chapters total
   Processing chapters: [1]

PROCESSING CHAPTER 1
   Years available: 2001 to 2024
   Initialized 13 chains for Year 2001

   Generating embeddings for chapter 1...
      Year 2001: 13 embeddings
      Year 2002: 11 embeddings
      Year 2003: 8 embeddings
      Year 2004: 9 embeddings
      Year 2005: 9 embeddings
      Year 2006: 10 embeddings
      Year 2007: 10 embeddings
      Year 2008: 10 embeddings
      Year 2009: 10 embeddings
      Year 2010: 9 embeddings
      Year 2011: 8 embeddings
      Year 2012: 8 embeddings
      Year 2014: 8 embeddings
      Year 2015: 8 embeddings
      Year 2017: 2 embeddings
      Year 2018: 2 embeddings
      Year 2019: 10 embeddings
      Year 2020: 1 embeddings
      Year 2021: 1 embeddings
      Year 2022: 9 embeddings
      Year 2023: 1 embeddings
      Year 2024: 9 embeddings

   Processing Chapter 1, Year 2002...
      Active chains: 13, Tables:

In [28]:
# Display in Colab
from IPython.display import IFrame
IFrame('output_chapter_1/sankey_chapter_1.html', width=800, height=600)

# Or download it
from google.colab import files
files.download('output_chapter_1/sankey_chapter_1.html')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [33]:
from google.colab import files
files.download('output_chapter_1/chains_chapter_1.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>